<a href="https://colab.research.google.com/github/ashishkhareiitr/projects/blob/master/YoLo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()



Saving data.zip to data.zip


In [0]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['data.zip']), 'r')
data.extractall()

In [34]:

_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]     


def loadImages(path):
    # return array of images
    
    imagesList = listdir(path)
    imagesList.sort(key=natural_sort_key)
    #imagesList = imagesList.sort()     
    loadedImages = []
    for image in imagesList:
      if image is not None:
        img = PIL.Image.open(path + image)
        loadedImages.append(img)
      else:
        print('not loaded')
    
    
    
    return loadedImages


# Specify the paths for the 2 files
#protoFile = "A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//pose_deploy_linevec.prototxt"
protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_440000.caffemodel"
 
path_loadImages =  "stickpose/"
imgs = loadImages(path_loadImages)


#path_poseImages = r"A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//poseImages//"

# Read the network into Memory
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

## GET FRAMES
#pathOut = r"A:/ISB/Capstone/PoseRetreival/FramePerSecond/"
#url = 'https://www.youtube.com/watch?v=AbPufvvYiSw'
#url='https://www.youtube.com/watch?v=SCr5mEl9qbQ'
#vPafy = pafy.new(url)
#play = vPafy.getbest(preftype="webm")

#Get from the video from local folder
vidcap = cv2.VideoCapture("yoga3.mp4")


#Get from the video from youtube
#vidcap = cv2.VideoCapture(play.url)


count = 0
success = True
'''
while success:
    success,image = vidcap.read()
    print('read a new frame:',success)
    if count%90 == 0 :
     #    cv2.imshow('test',image)
     #    cv2.imwrite(pathOut + 'frame%d.jpg'%count,image)
     #    #cv2.waitKey(1000)
     #    #cv2.destroyAllWindows()
    #count+=1
'''    


    
        
for i in range(1,len(imgs)): 
        frame=cv2.imread(imgs[i-1].filename)
        
        # Read image
        #frame = cv2.imread("A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//Image//Image3.jpg")
        
        #frame = image
        ###########Resize the image##################
         
        '''
        #basewidth = 1280
        #img = PIL.Image.open("A://ISB//Capstone//PoseRetreival//OpenCVPoseEsti//Image//Image3.jpg")
        width, height = img.size
        #print(width,height)
         
        wpercent = (basewidth/float(img.size[0]))
        hsize = int((float(img.size[1])*float(wpercent)))
        img = img.resize((basewidth,hsize), PIL.Image.ANTIALIAS)
        '''
        #width, height = frame.size
        height, width, channels = frame.shape
        print(width,height)
        #img.save('sompic.jpg') 
        if width <600:
            x=width
            y= 600 -x
            width = width +x
            height =  height +int((height/x)*y)
            frame = cv2.resize(frame, (width, height)) 
       
        ################################################
        # Read image
        #frame = cv2.imread(imgs[i-1].filename) #cv2.imread("sompic.jpg")
         
        # Specify the input image dimensions
        inWidth = width +0
        inHeight = height + int((height/width)*0)
        #print(inWidth,inHeight)
        # Prepare the frame to be fed to the network
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
         
        # Set the prepared object as the input blob of the network
        net.setInput(inpBlob)
        
        p=((0,1),(2,5),(3,6),(4,7),(8,11),(9,12),(10,13),(1,8),(1,11))        
        output = net.forward()
        
        H = output.shape[2]
        W = output.shape[3]
        # Empty list to store the detected keypoints
        points = {}
        for i in range(0,13):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]
        
            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
             
            # Scale the point to fit on the original image
            #x = (frameWidth * point[0]) / W
            #y = (frameHeight * point[1]) / H
            x = (inWidth * point[0]) / W
            y = (inHeight * point[1]) / H
            if prob > 0.01 : 
                cv2.circle(frame, (int(x), int(y)), 15, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frame, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 0, 255), 3, lineType=cv2.LINE_AA)
         
                # Add the point to the list if the probability is greater than the threshold
                points.update({i:(int(x), int(y))})
                #print(prob)
            else :
                #points.update(None)
                 print(points)
        
        for pair in p:
            partA = pair[0]
            partB = pair[1]
            print(partA,partB)
            #if points[partA] and points[partB]:
            if (partA in points) and (partB in points):
                cv2.line(frame, points[partA], points[partB], (0, 255, 0), 3)    
        
        cv2.imwrite(os.path.join(os.getcwd(),"stickpose{:d}.jpg".format(count)),frame )         
        #cv2.imshow("Output-Keypoints",frame)
        #cv2.waitKey(5000)
        #cv2.destroyAllWindows()
        count+=1

474 327
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
480 480
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
500 500
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
1000 667
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
1080 1108
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
480 480
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
960 720
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
500 333
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
640 640
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
720 810
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
570 310
{0: (310, 55), 1: (350, 111), 2: (406, 103), 3: (223, 190), 4: (167, 270), 5: (310, 119), 6: (223, 214), 7: (167, 270), 8: (374, 238), 9: (231, 87)}
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
500 500
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
400 300
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
768 1024
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
400 500
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
450 360
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
640 360
0 1
2 5
3 6
4 7
8 11
9 12
10 13
1 8
1 11
1280 720
0 1
2

In [3]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2018-11-11 11:31:46--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.3.39
Connecting to pjreddie.com (pjreddie.com)|128.208.3.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   108MB/s    in 2.2s    

2018-11-11 11:31:48 (108 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [0]:
#############################################
# Object detection - YOLO - OpenCV
# Author : Arun Ponnusamy   (July 16, 2018)
# Website : http://www.arunponnusamy.com
############################################


import cv2
import argparse
import numpy as np

import sys
import PIL.Image

import os
from sys import platform
import re
from os import listdir
import ntpath #for retrieving filename

import pandas as pd

#ap = argparse.ArgumentParser()
#ap.add_argument('-i', '--image', required=True,
#                help = 'path to input image')
#ap.add_argument('-c', '--config', required=True,
#                help = 'path to yolo config file')
#ap.add_argument('-w', '--weights', required=True,
#                help = 'path to yolo pre-trained weights')
#ap.add_argument('-cl', '--classes', required=True,
#                help = 'path to text file containing class names')
#args = ap.parse_args()


def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    image = cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return image

    
_nsre = re.compile('([0-9]+)')
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]     
def loadImages(path):
    # return array of images
    
    imagesList = listdir(path)
    imagesList.sort(key=natural_sort_key)
    #imagesList = imagesList.sort()     
    loadedImages = []
    for image in imagesList:
      if image is not None:
        img = PIL.Image.open(path + image)
        loadedImages.append(img)
      else:
        print('not loaded')
    
       
    return loadedImages



In [21]:
path_loadImages =  "backup data/train/8/"
count = 0
imgs = loadImages(path_loadImages)

for i in range(1,len(imgs)):
  
  print(imgs[i-1].filename)

  image=cv2.imread(imgs[i-1].filename)

#image = cv2.imread("frame328.jpg")

  Width = image.shape[1]
  Height = image.shape[0]
  scale = 0.00392

  classes = None

  with open("yolov3.txt", 'r') as f:
    
    classes = [line.strip() for line in f.readlines()]

  COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

  weights = "yolov3.weights"
  config = "yolov3.cfg"

  net = cv2.dnn.readNet(weights, config)

  blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

  net.setInput(blob)

  outs = net.forward(get_output_layers(net))

  class_ids = []
  confidences = []
  boxes = []
  conf_threshold = 0.5
  nms_threshold = 0.4


  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.5:
              center_x = int(detection[0] * Width)
              center_y = int(detection[1] * Height)
              w = int(detection[2] * Width)
              h = int(detection[3] * Height)
              x = center_x - w / 2
              y = center_y - h / 2
              class_ids.append(class_id)
              confidences.append(float(confidence))
              boxes.append([x, y, w, h])


  indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

  for i in indices:
      i = i[0]
      box = boxes[i]
      x = box[0]
      y = box[1]
      w = box[2]
      h = box[3]
      draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

  #cv2.imshow("object detection", image)
  #cv2.waitKey()

  #cv2.imwrite("object-detection.jpg", image)
  #cv2.destroyAllWindows()
  cv2.imwrite(os.path.join(os.getcwd(),"_8yolo{:d}.jpg".format(count)),image )
  
  xx = int(x)
  yy = int(y)
  ww = int(w)
  hh = int(h)
  
  img = cv2.imread("_8yolo{:d}.jpg".format(count))
  print(xx,yy,ww,hh)
  
  if img is not None:
    
    
    crop_img = img[yy:yy+hh,xx:xx+ww]
  #crop_img = img[604:700,420:900]
  #y:y+h, x:x+w]
  #cv2.imshow("cropped", crop_img)
    cv2.imwrite(os.path.join(os.getcwd(),"_8cropyolo{:d}.jpg".format(count)),crop_img)
  else:
    print("not cropped")
  
  count = count +1
  
  

backup data/train/8/500_F_175054143_TMoVXg7KGOHSiPlMfIYN2Gk0Ldty9S2T.jpg
15 40 178 116
backup data/train/8/frame4.jpg
105 56 30 198
backup data/train/8/frame5.jpg
3 -11 196 214
backup data/train/8/frame6.jpg
3 -10 197 212
backup data/train/8/frame44.jpg
254 30 70 307
backup data/train/8/frame45.jpg
255 32 70 305
backup data/train/8/frame46.jpg
255 31 70 307
backup data/train/8/frame47.jpg
254 31 71 312
backup data/train/8/frame155.jpg
394 129 66 175
backup data/train/8/frame232.jpg
366 35 78 280
backup data/train/8/frame233.jpg
366 40 78 270
backup data/train/8/frame292.jpg
163 39 108 287
backup data/train/8/frame293.jpg
162 39 109 287
backup data/train/8/frame294.jpg
162 39 109 287
backup data/train/8/frame425.jpg
135 332 41 44
backup data/train/8/frame436.jpg
135 388 39 44
backup data/train/8/frame437.jpg
137 392 37 42
backup data/train/8/frame438.jpg
136 397 39 43
backup data/train/8/frame641.jpg
706 30 208 689
backup data/train/8/frame642.jpg
704 33 208 683
backup data/train/8/fram

In [7]:
  import cv2
  cnt = 0
  img = cv2.imread("_2yolo{:d}.jpg".format(cnt)))
  print(x,y,w,h)
  
  if img is not None:
    
    
    crop_img = img[int(y):int(y+h),int(x):int(x+w)]
  #crop_img = img[604:700,420:900]
  #y:y+h, x:x+w]
  #cv2.imshow("cropped", crop_img)
    cv2.imwrite(os.path.join(os.getcwd(),"_2cropyolo{:d}.jpg".format(count)),crop_img)
  else:
    print("not cropped")
    
   cnt = cnt + 1
  

285.5 354.0 439 354


True